<a href="https://colab.research.google.com/github/haoran-zh/TransformerTrack-Colab_version/blob/main/TransformerTrack_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# connect google drive
# by running this, you can connect your code with your google drive.
# Upload your project files to your google drive in advance, then you can connect it with your code by running this.
from google.colab import drive
drive.mount('/content/drive') # the path where the drive is connected

# change the work directory
%cd /content/drive/MyDrive/[filename]

# check the work directory
!pwd

Mounted at /content/drive
/content/drive/MyDrive/TUMresearch
/content/drive/MyDrive/TUMresearch


In [ ]:
# detect GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# Tracker Model Preparation
1. Change working path.
2. Install necessary environments

In [ ]:
%cd /content/drive/MyDrive/[workpath]

/content/drive/MyDrive/TUMresearch


In [ ]:
# some packages you need before start
!bash install.sh conda_install_path pytracking
!pip install lvis
!pip install ipywidgets
!apt-get update
!apt-get install -y xvfb

# Training and testing

In [ ]:
%cd ltr

/content/drive/MyDrive/TUMresearch/TransformerTrack/ltr


In [ ]:
# fine-tune the pretrained model
import datetime
from google.colab import runtime

current_time = datetime.datetime.now()
print("Current time:", current_time)

!python run_training.py dimp transformer_dimp_ultrasound | tee ../../train_log.txt

current_time = datetime.datetime.now()
print("End time:", current_time)

runtime.unassign() # end the session after the training

In [ ]:
!pwd | tee ../../train_log.txt

/content


In [ ]:
import datetime
current_time = datetime.datetime.now()
print("Current time:", current_time)

Current time: 2023-05-31 22:11:06.984846


In [ ]:
%cd TransformerTrack/pytracking

/content/drive/MyDrive/TUMresearch/TransformerTrack/pytracking


In [ ]:
# test
# Create a virtual display to run the code in headless version
import subprocess
xvfb = subprocess.Popen(["Xvfb", ":99"])
import os
os.environ['DISPLAY'] = ':99'

!python run_video.py trdimp trdimp ../before0514/c_zhongliang_1.avi --optional_box 570 230 130 125 --debug 0 --save_results

xvfb.terminate() # end that

Using /root/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu118/_prroi_pooling/build.ninja...
Building extension module _prroi_pooling...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module _prroi_pooling...


# Show the result
add box on the original video

In [ ]:
!pwd

/content/drive/MyDrive/TUMresearch/TransformerTrack/pytracking


In [ ]:
# working path:
# Have the result of boxing.
import io
import base64
from IPython.display import HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import cv2
import os
from google.colab.patches import cv2_imshow # replace cv2.imshow()

def add_box(video_cv2, box_path):
  box_file = open(box_path, 'rb')
  box_text = box_file.readlines()

  box_text = [x.strip() for x in box_text] # remove '\n'
  box_list = []
  for line in box_text:
    row = line.split(bytes('\t', 'utf-8')) # remove '\t'
    row = [int(row_byte) for row_byte in row]
    box_list.append(row) # the box is like x y w h

  # Create output video writer
  frame_width = int(video_cv2.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(video_cv2.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = int(video_cv2.get(cv2.CAP_PROP_FPS))
  fourcc = cv2.VideoWriter_fourcc(*'MJPG')
  out = cv2.VideoWriter('../output_'+option[0:-4]+'.avi', fourcc, fps, (frame_width, frame_height))

  # process the video
  frame_index = 0
  while video_cv2.isOpened():
    ret, frame = video_cv2.read()
    if ret is False:
      break
    else:
      x_tl = box_list[frame_index][0]
      y_tl = box_list[frame_index][1]
      x_br = x_tl + box_list[frame_index][2]
      y_br = y_tl + box_list[frame_index][3]
      cv2.rectangle(frame, (x_tl, y_tl), (x_br, y_br), (0, 255, 0), 2)
      out.write(frame)
      frame_index += 1
      # cv2_imshow(frame)

  out.release()
  box_file.close()
  return 0



# Read the video file
# video_file = open('../original.avi', 'rb')
# video_bytes = video_file.read()

option = 'c_zhongliang_1.avi'
fv_path = os.path.join('../before0514', option)
video_cv2 = cv2.VideoCapture(fv_path)

frame_width = int(video_cv2.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_cv2.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_cv2.get(cv2.CAP_PROP_FPS))

video_boxed = add_box(video_cv2, box_path = 'tracking_results/trdimp/trdimp/video_'+option[0:-4]+'.txt')

# get the output_video
output_file = open('../output_'+option, 'rb')
output_bytes = output_file.read()

# Create the video player widget
video_widget = widgets.Video(
    value=output_bytes,
    format='avi',
    width=frame_width,
    height=frame_height,
    autoplay=True,
    loop=True
)

# Display the video player widget
video_cv2.release()
output_file.close()
# display(video_widget)